<a href="https://colab.research.google.com/github/rippedtuco/GenerativeAI/blob/main/PDF_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
!pip install langchain streamlit PyPDF2 python-dotenv faiss-cpu openai huggingface_hub import-ipynb
from google.colab import drive
from google.colab import files
drive.mount('/content/drive')
!python3 "/content/drive/My Drive/Colab Notebooks/htmlTemplate.ipynb"
#from htmlTemplate.ipynb import css, bot_template, user_template

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Traceback (most recent call last):
  File "/content/drive/My Drive/Colab Notebooks/htmlTemplate.ipynb", line 1, in <module>
    {"nbformat":4,"nbformat_minor":0,"metadata":{"colab":{"provenance":[],"authorship_tag":"ABX9TyNxodIaj/2gRSipAJi8tp82"},"kernelspec":{"name":"python3","display_name":"Python 3"},"language_info":{"name":"python"}},"cells":[{"cell_type":"code","execution_count":null,"metadata":{"id":"n1DI7Y1YAeFB"},"outputs":[],"source":["css = '''\n","<style>\n",".chat-message {\n","    padding: 1.5rem; border-radius: 0.5rem; margin-bottom: 1rem; display: flex\n","}\n",".chat-message.user {\n","    background-color: #2b313e\n","}\n",".chat-message.bot {\n","    background-color: #475063\n","}\n",".chat-message .avatar {\n","  width: 20%;\n","}\n",".chat-message .avatar img {\n","  max-width: 78px;\n","  max-height: 78px;\n","  border-radius: 50%;\n"," 

In [10]:
!pip install InstructorEmbedding sentence_transformers
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"]="XXXXXXXXXXXX"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 76.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 95.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 65.1 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=4d87f0d8bc106a577bbcb3a25292c94a1ce3d404cdddb0e7bf2dc42ed8251c22
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [16]:
%%writefile app.py

import streamlit as st
import langchain
from dotenv import load_dotenv
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFaceHub

def get_Pdf_Text(pdf_docs):
    text=""
    for pdf in pdf_docs:
        pdf_reader = PdfReader(pdf)
        for page in pdf_reader.pages:
            text +=page.extract_text()
    return text

def get_Text_Chunks(text):
    text_splitter= CharacterTextSplitter(separator="\n", chunk_size=1000,chunk_overlap=200,length_function=len)
    chunks=text_splitter.split_text(text)
    return chunks

def get_VectorStore(chunks):
    embeddings=HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-xl")
    vectorstore=FAISS.from_texts(texts=chunks, embedding=embeddings)
    return vectorstore

def get_Conversation_Chain(vectorstore):
    llm = HuggingFaceHub(repo_id="google/flan-t5-xxl", model_kwargs={"temperature":0.5, "max_length":512})
    memory=ConversationBufferMemory(memory_key='chat_history', return_messages=True)
    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm=llm,
        retriever=vectorstore.as_retriever(),
        memory=memory
    )
    return conversation_chain

def handle_userinput(user_question):
    response = st.session_state.conversation({'question': user_question})
    st.session_state.chat_history = response['chat_history']

    for i, message in enumerate(st.session_state.chat_history):
        if i % 2 == 0:
            st.write(user_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)
        else:
            st.write(bot_template.replace(
                "{{MSG}}", message.content), unsafe_allow_html=True)

def main():
    load_dotenv()
    st.set_page_config(page_title="Chat with Multiple PDF's", page_icon=":biceps:")

    if "conversation" not in st.session_state:
        st.session_state.conversation = None

    st.header("Chat with!")
    st.text_input("Ask!")

    with st.sidebar:
        st.subheader("Your docs")
        pdf_files= st.file_uploader("Upload your files",accept_multiple_files=True)
        if st.button("Process"):
            with st.spinner("Processing"):
                raw_text=get_Pdf_Text(pdf_files)
                text_chunks=get_Text_Chunks(raw_text)
                vectorstore = get_VectorStore(text_chunks)
                convo = get_Conversation_Chain(vectorstore)
                st.session_state.conversation= get_Conversation_Chain(vectorstore)


if __name__=='__main__':
    main()

Overwriting app.py


In [17]:
!npm install localtunnel

npm WARN saveError ENOENT: no such file or directory, open '/content/package.json'
npm WARN enoent ENOENT: no such file or directory, open '/content/package.json'
npm WARN content No description
npm WARN content No repository field.
npm WARN content No README data
npm WARN content No license field.

+ localtunnel@2.0.2
updated 1 package and audited 36 packages in 0.536s

3 packages are looking for funding
  run `npm fund` for details

found 0 vulnerabilities



In [18]:
!streamlit run app.py &>/content/logs.txt &

In [19]:
!npx localtunnel --port 8501 & curl ipv4.icanhazip.com

104.196.36.23
npx: installed 22 in 1.394s
your url is: https://thin-sheep-appear.loca.lt
